In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import xarray as xr
from IPython.display import display

In [2]:
# load wildfire dataset
df = pd.read_csv("wildfires.csv")
print("loaded df")

loaded df


C:\Users\jepoisso\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# define the dataset settings
mnet_datasets = [
    #{
    #    'name': 'pdsi',
    #    'col': 'palmer_drought_severity_index',
    #    'ops': ['min', 'max', 'avg', 'count']
    #},
    {
        'name': 'pr',
        'col': 'precipitation_amount',
        'ops': ['min', 'max', 'avg', 'count']
    },
    {
        'name': 'rmax',
        'col': 'relative_humidity',
        'ops': ['min', 'max', 'avg', 'count']
    },
    {
        'name': 'rmin',
        'col': 'relative_humidity',
        'ops': ['min', 'max', 'avg', 'count']
    },
    {
        'name': 'tmmn',
        'col': 'air_temperature',
        'ops': ['min', 'max', 'avg', 'count']
    },
    {
        'name': 'tmmx',
        'col': 'air_temperature',
        'ops': ['min', 'max', 'avg', 'count']
    },
    {
        'name': 'vs',
        'col': 'air_temperature',
        'ops': ['min', 'max', 'avg', 'count']
    }
]
# constant values
jan1st1900 = datetime(1900, 1, 1)
progress_count = 100

In [4]:
# main function for executing the different operations for each series
def execute_op(op, entries_df):
    l = len(entries_df)
    # optimization: xarray.sel is inclusive, hence 2 entries is essentially one entry
    if op == 'min':
        return entries_df.min() if l > 2 else entries_df.iloc[0]
    elif op == 'max':
        return entries_df.max() if l > 2 else entries_df.iloc[0]
    elif op == 'avg':
        return entries_df.mean() if l > 2 else entries_df.iloc[0]
    elif op == 'count':
        return l
    return np.nan

In [6]:
# process the data & output progress
years = df['FIRE_YEAR'].unique()
for y in years:
    # go through each dataset per year
    for _, ds_metadata in enumerate(mnet_datasets):
        mnet_ds_filename = 'mnet/%s_%d.nc' % (ds_metadata['name'], y)
        mnet_ds = xr.open_dataset(mnet_ds_filename, decode_cf=False)
        time_start = datetime.now().timestamp()
        processed_count, invalid_count = 0, 0
        
        df_y = df[df['FIRE_YEAR'] == y]
        print("opened up %s" % mnet_ds_filename)
        dh = display('opened up %s' % mnet_ds_filename, display_id=True)
        # loop thru rows 
        for index, row in df_y.iterrows():
            #print("> processing index %d for year %d and dataset %s" % (index, y, mnet_ds_filename))
            # date from year + day of year
            f_year, disc_doy, cont_doy, latitude, longitude = row['FIRE_YEAR'], row['DISCOVERY_DOY'], row['CONT_DOY'], row['LATITUDE'], row['LONGITUDE']
            if np.isnan(f_year) or np.isnan(disc_doy) or np.isnan(latitude) or np.isnan(longitude):
                invalid_count += 1
                print("\tskipping invalid index %d" % (index), f_year, disc_doy, cont_doy, latitude, longitude)
                continue
                
            d_start = datetime(f_year, 1, 1) + timedelta(days=np.float64(disc_doy))
            d_end = datetime(f_year, 1, 1) + (timedelta(days=np.float64(cont_doy)) if not np.isnan(cont_doy) else timedelta(days=np.float64(disc_doy)))
            delta_start = (d_start - jan1st1900).days
            delta_end = (d_end - jan1st1900).days
            lat = np.float64(latitude)
            lon = np.float64(longitude)
            # select relevant entries in mnet_dataset
            mnet_entries = mnet_ds.sel(day=[delta_start, delta_end], lon=lon, lat=lat, method="nearest")
            mnet_df = mnet_entries.to_dataframe()
            # compute operations for entries and store in augment_cols
            augment_cols = dict()
            for op in ds_metadata['ops']:
                col_key = "%s_%s" % (ds_metadata['name'], op)
                augment_cols[col_key] = execute_op(op, mnet_df[ds_metadata['col']])
                # update dataset with new column
                df.at[index, col_key] = augment_cols[col_key]
            processed_count += 1    
            #print("\tentry %d : %d mnet entries in %s: %s" % (index, mnet_df.shape[0], mnet_ds_filename, augment_cols))
            if processed_count % progress_count == 0:
                percentage = (processed_count + invalid_count) / df_y.shape[0]
                dh.update("[%d] '%s' dataset progress: %d / %d [%.2f] (%d invalids)" % (datetime.now().timestamp(), mnet_ds_filename, (processed_count + invalid_count), df_y.shape[0], percentage, invalid_count))
        time_end = datetime.now().timestamp()
        time_delta = time_end - time_start
        print("finished with %s, processed = %d, invalid = %d : took %d seconds" % (mnet_ds_filename, processed_count, invalid_count, time_delta))
    
output_f = "wildfire_augmented_%d.csv" % datetime.now().timestamp() 
print("done processing entries! output is %s" % output_f)
df.to_csv(output_f)

opened up mnet/pdsi_2005.nc


"[1606584785] 'mnet/pdsi_2005.nc' dataset progress: 88600 / 88604 [1.00] (0 invalids)"

finished with mnet/pdsi_2005.nc, processed = 88604, invalid = 0 : took 341 seconds
opened up mnet/pdsi_2004.nc


"[1606585035] 'mnet/pdsi_2004.nc' dataset progress: 69200 / 69279 [1.00] (0 invalids)"

finished with mnet/pdsi_2004.nc, processed = 69279, invalid = 0 : took 250 seconds
opened up mnet/pdsi_2006.nc


...

finished with mnet/pdsi_2013.nc, processed = 64780, invalid = 0 : took 344 seconds
opened up mnet/pdsi_2014.nc


"[1606606328] 'mnet/pdsi_2014.nc' dataset progress: 67700 / 67753 [1.00] (0 invalids)"

finished with mnet/pdsi_2014.nc, processed = 67753, invalid = 0 : took 401 seconds
opened up mnet/pdsi_2015.nc


"[1606606735] 'mnet/pdsi_2015.nc' dataset progress: 74400 / 74491 [1.00] (0 invalids)"

finished with mnet/pdsi_2015.nc, processed = 74491, invalid = 0 : took 407 seconds
done processing entries! output is wildfire_augmented_1606606735.csv


In [12]:
ds = xr.open_dataset("mnet/vs_1992.nc", decode_cf=False)
ds

<xarray.Dataset>
Dimensions:     (crs: 1, day: 366, lat: 585, lon: 1386)
Coordinates:
  * lon         (lon) float64 -124.8 -124.7 -124.7 ... -67.14 -67.1 -67.06
  * lat         (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * day         (day) float64 3.36e+04 3.36e+04 3.36e+04 ... 3.397e+04 3.397e+04
  * crs         (crs) uint16 3
Data variables:
    wind_speed  (day, lat, lon) uint16 ...
Attributes:
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    geospatial_lon_max:         -67.058333300000015
    geospatial_lon_resolution:  0.041666666666666
    geospatial_lat_resolution:  0.041666666666666
    geospatial_lat_units:       decimal_degrees north
    geospatial_lon_units:       decimal_degrees east
    coordinate_system:          EPSG:4326
    author:                     John Abatzoglou - University of Idaho, jabatz...
    date:                       03 July 2019
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...